# Day 16 - o1

In [1]:
import heapq

def read_maze(filename):
    with open(filename, 'r') as f:
        lines = [line.rstrip('\n') for line in f]
    return lines

def find_positions(maze):
    start = None
    end = None
    for r, line in enumerate(maze):
        for c, ch in enumerate(line):
            if ch == 'S':
                start = (r, c)
            elif ch == 'E':
                end = (r, c)
    return start, end

def solve():
    maze = read_maze('input.txt')
    rows = len(maze)
    cols = len(maze[0]) if rows > 0 else 0
    
    start, end = find_positions(maze)
    # Directions: 0 = East, 1 = South, 2 = West, 3 = North
    # dx, dy in order East, South, West, North
    directions = [(0,1),(1,0),(0,-1),(-1,0)]
    
    # Dijkstra’s algorithm
    # State: (cost, row, col, dir)
    # Start facing East (dir = 0)
    start_state = (0, start[0], start[1], 0)
    dist = [[[float('inf')] * 4 for _ in range(cols)] for __ in range(rows)]
    dist[start[0]][start[1]][0] = 0
    
    pq = []
    heapq.heappush(pq, start_state)
    
    while pq:
        cost, r, c, d = heapq.heappop(pq)
        if cost > dist[r][c][d]:
            continue
        # Check if we reached the end
        if (r, c) == end:
            print(cost)
            return
        
        # Try moving forward
        dr, dc = directions[d]
        nr, nc = r + dr, c + dc
        if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] != '#':
            # Move forward cost 1
            new_cost = cost + 1
            if new_cost < dist[nr][nc][d]:
                dist[nr][nc][d] = new_cost
                heapq.heappush(pq, (new_cost, nr, nc, d))
        
        # Try rotating left and right
        # Left turn: (d - 1) % 4
        # Right turn: (d + 1) % 4
        left_dir = (d - 1) % 4
        right_dir = (d + 1) % 4
        
        # Rotate left (cost 1000)
        new_cost = cost + 1000
        if new_cost < dist[r][c][left_dir]:
            dist[r][c][left_dir] = new_cost
            heapq.heappush(pq, (new_cost, r, c, left_dir))
        
        # Rotate right (cost 1000)
        new_cost = cost + 1000
        if new_cost < dist[r][c][right_dir]:
            dist[r][c][right_dir] = new_cost
            heapq.heappush(pq, (new_cost, r, c, right_dir))

# Run the solver
if __name__ == "__main__":
    solve()


66404


## Part 2

In [2]:
import heapq

def read_maze(filename):
    with open(filename, 'r') as f:
        lines = [line.rstrip('\n') for line in f]
    return lines

def find_positions(maze):
    start = None
    end = None
    for r, line in enumerate(maze):
        for c, ch in enumerate(line):
            if ch == 'S':
                start = (r, c)
            elif ch == 'E':
                end = (r, c)
    return start, end

def solve():
    maze = read_maze('input.txt')
    rows = len(maze)
    cols = len(maze[0]) if rows > 0 else 0
    
    start, end = find_positions(maze)
    # Directions: 0 = East, 1 = South, 2 = West, 3 = North
    directions = [(0,1),(1,0),(0,-1),(-1,0)]

    # Run Dijkstra to find shortest path cost
    dist = [[[float('inf')] * 4 for _ in range(cols)] for __ in range(rows)]
    dist[start[0]][start[1]][0] = 0
    pq = []
    heapq.heappush(pq, (0, start[0], start[1], 0))  # cost, r, c, direction
    
    while pq:
        cost, r, c, d = heapq.heappop(pq)
        if cost > dist[r][c][d]:
            continue
        if (r, c) == end:
            # Not returning here because we need full dist array for part two
            pass
        
        # Move forward
        dr, dc = directions[d]
        nr, nc = r + dr, c + dc
        if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] != '#':
            new_cost = cost + 1
            if new_cost < dist[nr][nc][d]:
                dist[nr][nc][d] = new_cost
                heapq.heappush(pq, (new_cost, nr, nc, d))
        
        # Rotate left
        left_d = (d - 1) % 4
        new_cost = cost + 1000
        if new_cost < dist[r][c][left_d]:
            dist[r][c][left_d] = new_cost
            heapq.heappush(pq, (new_cost, r, c, left_d))
        
        # Rotate right
        right_d = (d + 1) % 4
        new_cost = cost + 1000
        if new_cost < dist[r][c][right_d]:
            dist[r][c][right_d] = new_cost
            heapq.heappush(pq, (new_cost, r, c, right_d))
    
    # Find minimal cost to reach the end tile in any direction
    best_cost = min(dist[end[0]][end[1]])
    
    # Backward search to find all states on a best path
    # We'll do a queue-based reverse search
    # Start from all end states that achieve best_cost
    best_end_states = []
    for d in range(4):
        if dist[end[0]][end[1]][d] == best_cost:
            best_end_states.append((end[0], end[1], d))
    
    on_best_path = set(best_end_states)
    from_collections = []
    for st in best_end_states:
        from_collections.append(st)
    
    # We'll check all possible predecessors of a given state
    # Predecessor moves:
    # For forward: if (nr,nc,d) -> (r,c,d) was forward, then (r,c,d) must be (nr-dr, nc-dc, d)
    # For rotation: if (r,c,d2) -> (r,c,d) was rotation, then (r,c,d2) is predecessor
    # Cost checks:
    # forward move cost = 1
    # rotation cost = 1000
    
    # We'll keep checking until no new states are found
    while from_collections:
        r, c, d = from_collections.pop()
        cur_cost = dist[r][c][d]
        
        # Check rotation predecessors:
        # If current state direction is d, predecessor could have rotated right or left to get here
        # Predecessors:
        # left rotation: (r,c,(d+1)%4) -> (r,c,d) cost:1000
        # right rotation: (r,c,(d-1)%4) -> (r,c,d) cost:1000
        left_pred = (d + 1) % 4
        if 0 <= r < rows and 0 <= c < cols:
            if dist[r][c][left_pred] + 1000 == cur_cost:
                if (r,c,left_pred) not in on_best_path:
                    on_best_path.add((r,c,left_pred))
                    from_collections.append((r,c,left_pred))
        
        right_pred = (d - 1) % 4
        if dist[r][c][right_pred] + 1000 == cur_cost:
            if (r,c,right_pred) not in on_best_path:
                on_best_path.add((r,c,right_pred))
                from_collections.append((r,c,right_pred))
        
        # Check forward predecessor:
        # If we got here by moving forward in direction d,
        # then the predecessor is at (r - dr, c - dc, d) with cost cur_cost - 1
        dr, dc = directions[d]
        pr, pc = r - dr, c - dc
        if 0 <= pr < rows and 0 <= pc < cols and maze[pr][pc] != '#':
            # Check if dist[pr][pc][d] + 1 == dist[r][c][d]
            if dist[pr][pc][d] + 1 == cur_cost:
                if (pr,pc,d) not in on_best_path:
                    on_best_path.add((pr,pc,d))
                    from_collections.append((pr,pc,d))
    
    # Now we have a set of all states that are on a best path.
    # Extract unique tile positions (r,c) that are part of best paths.
    best_path_tiles = set((r,c) for (r,c,d) in on_best_path)
    
    # Count how many tiles are on at least one best path
    # Only count non-wall tiles. S, E, . are valid seating positions.
    count = 0
    for (rr,cc) in best_path_tiles:
        if maze[rr][cc] != '#':
            count += 1
    
    print(count)

if __name__ == "__main__":
    solve()


433
